# Robo Advisor - Dashboard

## 1. Loading the libraries and the data

In [97]:
import dash
import dash_core_components as dcc
import dash_html_components as html
from jupyter_dash import JupyterDash
from dash.dependencies import Input, Output, State
import pandas as pd
import numpy as np
import plotly.graph_objs as go
import dash_daq as daq
from pickle import load
import cvxopt as opt
from cvxopt import blas, solvers

In [98]:
investors = pd.read_csv('data/InputData.csv', index_col=0)
investors.head()

,AGE07,EDCL07,MARRIED07,KIDS07,LIFECL07,OCCAT107,INCOME07,RISK07,WSAVED07,SPENDMOR07,NETWORTH07,TrueRiskTol
0,47,2,1,0,2,1,56443.744181,3,1,5,352641.71130,6.947439
1,47,2,1,0,2,1,56443.744181,3,1,5,340525.21881,6.223158
2,47,2,1,0,2,1,56443.744181,3,1,5,348696.08426,6.676709
3,47,2,1,0,2,1,56443.744181,3,1,5,351802.87720,6.286854
4,47,2,1,0,2,1,56443.744181,3,1,5,358958.85696,6.177016


In [99]:
assets = pd.read_csv('data/SP500Data.csv', index_col=0)
assets.head()

,ABT,ABBV,ABMD,ACN,ATVI,ADBE,AMD,AAP,AES,AMG,...,WLTW,WYNN,XEL,XRX,XLNX,XYL,YUM,ZBH,ZION,ZTS
Date,,,,,,,,,,,,,,,,,,,,,
2018-01-02,58.790001,98.410004,192.490005,153.839996,64.309998,177.699997,10.98,106.089996,10.88,203.039993,...,146.990005,164.300003,47.810001,29.370001,67.879997,68.070000,81.599998,124.059998,50.700001,71.769997
2018-01-03,58.919998,99.949997,195.820007,154.550003,65.309998,181.039993,11.55,107.050003,10.87,202.119995,...,149.740005,162.520004,47.490002,29.330000,69.239998,68.900002,81.529999,124.919998,50.639999,72.099998
2018-01-04,58.820000,99.379997,199.250000,156.380005,64.660004,183.220001,12.12,111.000000,10.83,198.539993,...,151.259995,163.399994,47.119999,29.690001,70.489998,69.360001,82.360001,124.739998,50.849998,72.529999
2018-01-05,58.990002,101.110001,202.320007,157.669998,66.370003,185.339996,11.88,112.180000,10.87,199.470001,...,152.229996,164.490005,46.790001,29.910000,74.150002,69.230003,82.839996,125.980003,50.869999,73.360001
2018-01-08,58.820000,99.489998,207.800003,158.929993,66.629997,185.039993,12.28,111.389999,10.87,200.529999,...,151.410004,162.300003,47.139999,30.260000,74.639999,69.480003,82.980003,126.220001,50.619999,74.239998


In [100]:
missing_fractions = assets.isnull().mean().sort_values(ascending=False)
missing_fractions.head()

CTVA    0.781250
DOW     0.678571
FOX     0.667411
FOXA    0.665179
NOV     0.000000
dtype: float64

In [101]:
drop_list = sorted(list(missing_fractions[missing_fractions > 0.3].index))
print(drop_list)

['CTVA', 'DOW', 'FOX', 'FOXA']


In [102]:
assets.drop(labels=drop_list, axis=1, inplace=True)
assets.shape
# Fill the missing values with the last value available in the dataset.
assets=assets.fillna(method='ffill')
assets.head()

,ABT,ABBV,ABMD,ACN,ATVI,ADBE,AMD,AAP,AES,AMG,...,WLTW,WYNN,XEL,XRX,XLNX,XYL,YUM,ZBH,ZION,ZTS
Date,,,,,,,,,,,,,,,,,,,,,
2018-01-02,58.790001,98.410004,192.490005,153.839996,64.309998,177.699997,10.98,106.089996,10.88,203.039993,...,146.990005,164.300003,47.810001,29.370001,67.879997,68.070000,81.599998,124.059998,50.700001,71.769997
2018-01-03,58.919998,99.949997,195.820007,154.550003,65.309998,181.039993,11.55,107.050003,10.87,202.119995,...,149.740005,162.520004,47.490002,29.330000,69.239998,68.900002,81.529999,124.919998,50.639999,72.099998
2018-01-04,58.820000,99.379997,199.250000,156.380005,64.660004,183.220001,12.12,111.000000,10.83,198.539993,...,151.259995,163.399994,47.119999,29.690001,70.489998,69.360001,82.360001,124.739998,50.849998,72.529999
2018-01-05,58.990002,101.110001,202.320007,157.669998,66.370003,185.339996,11.88,112.180000,10.87,199.470001,...,152.229996,164.490005,46.790001,29.910000,74.150002,69.230003,82.839996,125.980003,50.869999,73.360001
2018-01-08,58.820000,99.489998,207.800003,158.929993,66.629997,185.039993,12.28,111.389999,10.87,200.529999,...,151.410004,162.300003,47.139999,30.260000,74.639999,69.480003,82.980003,126.220001,50.619999,74.239998


In [103]:
options = []
for tic in assets.columns:
    mydict = {}
    mydict['label'] = tic
    mydict['value'] = tic
    options.append(mydict)

## 2. Code for the dashboard interface

In [104]:
app = JupyterDash(__name__, external_stylesheets=['https://codepen.io/chriddyp/pen/bWLwgP.css'])
#app = dash.Dash(__name__, external_stylesheets=['https://codepen.io/chriddyp/pen/bWLwgP.css'])

In [105]:
app.layout = html.Div([
    html.Div([
        # Dashboard name
        html.Div([
            html.H3(children='Robo Advisor Dashboard'),
            html.Div([
                html.H5(children='Step 1: Enter Investor Characteristics'),
                ], style={'display':'inline-block', 'vertical-align':'top', 'width':'30%', 'color':'black', 'background-color':'LightGray'}),
            html.Div([
                html.H5(children='Step 2: Asset Allocation and Portfolio Performance'),
                ], style={'display':'inline-block', 'vertical-align':'top', 'color':'white', 'horizontalAlign':'left', 'width':'70%', 'background-color':'black'}),
            ], style={'font-family':'calibri'}),

        # All the investor characteristics
        # Demographic features
        html.Div([
            html.Div([
                html.Label('Age: ', style={'padding':5}),
                dcc.Slider(
                    id = 'Age',
                    min = investors['AGE07'].min(),
                    max = 70,
                    marks = {25:'25', 35:'35', 45:'45', 55:'55', 70:'70'},
                    value = 25),

                html.Label('NetWorth: ', style={'padding':5}),
                dcc.Slider(
                    id = 'Nwcat',
                    min = -1000000,
                    max = 3000000,
                    marks = {-1000000:'-$1M', 0:'0', 500000:'$500K', 1000000:'$1M', 2000000:'$2M'},
                    value = 10000),
                            
                html.Label('Education Level (scale of 4):', style={'padding':5}),
                dcc.Slider(
                    id = 'Edu',
                    min = investors['EDCL07'].min(), 
                    max = investors['EDCL07'].max(),
                    marks = {1:'1', 2:'2', 3:'3', 4:'4'},
                    value = 2), 

                html.Label('Married: ', style={'padding':5}),
                dcc.Slider(
                    id = 'Married',
                    min = investors['MARRIED07'].min(), 
                    max = investors['MARRIED07'].max(),
                    marks = {1:'1', 2:'2'},
                    value = 1),

                html.Label('Kids: ', style={'padding':5}),
                dcc.Slider(
                    id = 'Kids',
                    min = investors['KIDS07'].min(), 
                    max = investors['KIDS07'].max(),
                    marks = {1: '1', 2:'2', 3:'3', 4:'4'},
                    #marks = [{'label':j, 'value':j} for j in investors['KIDS07'].unique()],
                    value = 3), 

                html.Label('Occupation: ', style={'padding':5}),
                dcc.Slider(
                    id = 'Occ',
                    min = investors['OCCAT107'].min(), 
                    max = investors['OCCAT107'].max(),
                    marks = {1:'1', 2:'2', 3:'3', 4:'4'},
                    value = 3),            

                html.Label('Willingness to take Risk: ', style={'padding':5}),
                dcc.Slider(
                    id = 'Risk',
                    min = investors['RISK07'].min(), 
                    max = investors['RISK07'].max(),
                    marks = {1:'1', 2:'2', 3:'3', 4:'4'},
                    value = 3), 

                html.Button(
                    id = 'investor_char_button',
                    n_clicks = 0,
                    children = 'Calculate Rist Tolerance',
                    style = {'fontSize':14, 'marginLeft':'30px', 'color':'white', 'horizontal-align':'left', 'backgroundColor':'grey'}
                ),
            ], style={'width':'80%'}),
        ], style={'width':'30%', 'font-family':'calibri', 'vertical-align':'top', 'display':'inline-block'}),

        # Risk Tolerance charts
        html.Div([
            html.Div([
                html.Label('Risk Tolerance (scale of 100): ', style={'padding':5}),
                dcc.Input(id='risk-tolerance-text'),
            ], style={'width':'100%', 'font-family':'calibri', 'vertical-align':'top', 'display':'inline-block'}),        
            
            html.Div([
                html.Label('Select the assets for the portfolio: ', style={'padding':5}),
                dcc.Dropdown(
                    id = 'ticker_symbol',
                    options = options,
                    value = ['GOOGL', 'FB', 'GS', 'MS', 'GE', 'MSFT'],
                    multi = True,
                ),
                html.Button(
                    id = 'submit-asset_alloc_button',
                    n_clicks = 0,
                    children = 'Submit',
                    style = {'fontSize':12, 'marginLeft':'25px', 'color':'white', 'backgroundColor':'grey'}
                )
            ], style={'width':'100%', 'display':'inline-block', 'font-family':'calibri', 'vertical-align':'top'}),

            html.Div([
                html.Div([
                    dcc.Graph(id='asset-alloc'),
                ], style={'width':'50%', 'vertical-align':'top', 'display':'inline-block', 'font-family':'calibri', 'horizontal-align':'right'}),
                html.Div([
                    dcc.Graph(id='performance'),
                ], style={'width':'50%', 'vertical-align':'top', 'display':'inline-block', 'font-family':'calibri', 'horizontal-align':'right'}),
            ], style={'width':'100%', 'vertical-align':'top', 'display':'inline-block', 'font-family':'calibri', 'horizontal-align':'right'}),
        ], style={'width':'70%', 'vertical-align':'top', 'display':'inline-block', 'font-family':'calibri', 'horizontal-align':'right'})
    
    ], style={'width':'80%', 'vertical-align':'top', 'display':'inline-block', 'font-family':'calibri'})
])

In [106]:
if __name__ == '__main__':
    app.run_server(debug=True)
    #app.run_server(mode='jupyterlab', debug=True)

/Users/jkwon/opt/anaconda3/envs/test/lib/python3.8/site-packages/jupyter_dash/jupyter_app.py:139: UserWarning: The 'environ['werkzeug.server.shutdown']' function is deprecated and will be removed in Werkzeug 2.1.
  func()


Dash app running on http://127.0.0.1:8050/
